In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn
import csv
import json
import nltk
import os
import random
import re
import torch
from torch import nn, optim
import torch.nn.functional as F

In [0]:
train_csv_path = 'drive/My Drive/Colab Notebooks/DrugData/train_original.csv'

In [3]:
train_df = pd.read_csv(train_csv_path, header=0)
train_df.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [4]:
train_df.describe()

,sentiment
count,5279.000000
mean,1.607691
std,0.687203
min,0.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


In [5]:
sentiment_2_set = train_df[['text', 'drug', 'sentiment']][train_df['sentiment'] == 2]
sentiment_1_set = train_df[['text', 'drug', 'sentiment']][train_df['sentiment'] == 1]
sentiment_0_set = train_df[['text', 'drug', 'sentiment']][train_df['sentiment'] == 0]
print(len(sentiment_2_set))
print(len(sentiment_1_set))
print(len(sentiment_0_set))

3825
837
617


In [0]:
def sample_data_from_df(df, frac):
  if frac <= 1.0:
    temp_df = df.sample(frac = frac, replace=False)
    return temp_df
  else:
    temp_df = df.sample(frac = frac, replace=True)
    return temp_df
  

In [0]:
rows_needed_for_0 = 900
rows_needed_for_1 = 1200
rows_needed_for_2 = 1000
sentiment_2_set = sample_data_from_df(sentiment_2_set, rows_needed_for_2/len(sentiment_2_set))
sentiment_1_set = sample_data_from_df(sentiment_1_set, rows_needed_for_1/len(sentiment_1_set))
sentiment_0_set = sample_data_from_df(sentiment_0_set, rows_needed_for_2/len(sentiment_0_set))

total_train_df = pd.concat([sentiment_2_set,sentiment_1_set,sentiment_0_set])
total_train_df = total_train_df.sample(frac = 1, replace=False)

In [8]:
len(total_train_df)

3200

In [0]:
all_texts = []

In [10]:
'''with open(train_csv_path, 'r', encoding='utf-8') as f:
  input_data = csv.reader(f)
  
  for row in input_data:
    all_texts.append(row)'''

all_texts = []
for _, row in total_train_df.iterrows():
  all_texts.append(row)
  
print(len(all_texts))

3200


In [0]:
messages = [text[0] for text in all_texts]
sentiments = [text[2] for text in all_texts]
drug_info = [text[1] for text in all_texts]

In [0]:
all_drug_info = []
for index, item in train_df[['drug']].iterrows():
  all_drug_info.append(item[0].replace(' ', '-'))

In [0]:
index = 0
for message in messages:
  drug_name_without_space = drug_info[index].replace(' ', '-')
  messages[index] = message.replace(drug_info[index], drug_name_without_space)
  drug_info[index] = drug_name_without_space
  index += 1

In [14]:
print(messages[0])
print(sentiments[:20])
print(drug_info[:10])

hello! i hope you are doing well. my daughter had optic neuritis and was hospitalized in April 2018. her neurologist is recommending two drugs, either Copaxone and Ocrevus.. and we are unsure of what to do. Can you please tell me what course of medication you are on now? Good luck to you.
[1, 2, 0, 0, 2, 1, 0, 2, 2, 2, 2, 1, 1, 1, 2, 2, 0, 0, 2, 1]
['ocrevus', 'gilenya', 'ocrelizumab', 'keytruda', 'tarceva', 'ocrelizumab', 'ocrelizumab', 'gilenya', 'entyvio', 'ocrevus']


In [15]:
#0 - positive
#1 - negative
#2 - neutral
from collections import Counter
sentiment_counter = Counter(sentiments)
sentiment_counter.most_common()

[(1, 1200), (2, 1000), (0, 1000)]

In [16]:
drugs_set = set(all_drug_info)
print(drugs_set)
print(len(drugs_set)) #original - 102

{'mavenclad', 'atezolizumab', 'crizotinib', 'gefitinib', 'giotrif', 'tofacitinib', 'necitumumab', 'guselkumab', 'fingolimod', 'brigatinib', 'panretinal-photocoagulation', 'tafinlar', 'nivolumabb', 'osimertinib', 'ceritinib', 'certolizumab-pegol', 'ipilimumab', 'brolucizumab', 'rhumab-2h7', 'durvalumab', 'entrectinib', 'laser-photocoagulation', 'vedolizumab', 'simponi', 'arzerra', 'ct-p13', 'alectinib', 'alectnib', 'tagrisso', 'cladribine', 'trametinib', 'amjevita', 'alunbrig', 'movectro', 'tarceva', 'ozurdex', 'alecensa', 'tecentriq', 'alemtuzumab', 'pegaptanib', 'ustekinumab', 'opdivo', 'renflexis', 'lucentis', 'ofatumumab', 'pembrolizumab', 'zykadia', 'avastin', 'mekinist', 'vitrectomy', 'stelara', 'ocrevus', 'pemetrexed', 'imfinzi', 'keytruda', 'gilotrif', 'eylea', 'yervoy', 'erlotinib', 'golimumab', 'pemrolizumab', 'dabrafenib', 'cyramza', 'nivolumab', 'xalkori', 'remicade', 'filgotinib', 'lorlatinib', 'siponimod', 'ranibizumab', 'bevacizumab', 'cimzia', 'ixifi', 'macugen', 'terifl

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
!pip install pytorch-transformers

In [19]:
!pip install pytorch_pretrained_bert

In [0]:
#from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import os
from torch.utils.data import Dataset, DataLoader

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
text = 'all the world is a stage'
temp_tokens = tokenizer.tokenize(text)
print(temp_tokens)

['all', 'the', 'world', 'is', 'a', 'stage']


In [0]:
class BertLayerNorm(nn.Module):
  def __init__(self, hidden_size, eps=1e-12):
    super(BertLayerNorm, self).__init__()
    self.weight = nn.Parameter(torch.ones(hidden_size))
    self.bias = nn.Parameter(torch.zeros(hidden_size))
    self.variance_epsilon = eps
    
  def forward(self, x):
    u = x.mean(-1, keepdim=True)
    s = (x - u).pow(2).mean(-1, keepdim=True)
    x = (x - u)/ torch.sqrt(s + self.variance_epsilon)
    return self.weight * x + self.bias

In [0]:
class BertForSequenceClassification(nn.Module):
  def __init__(self, num_labels = 2):
    super(BertForSequenceClassification, self).__init__()
    self.num_labels = num_labels
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    #self.bert = BertModel.from_pretrained('bert-large-uncased')
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, num_labels)
    
    nn.init.xavier_normal_(self.classifier.weight)
    
  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
    _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)
    
    return logits
    
  def freeze_bert_encoder(self):
    for param in self.bert.parameters():
      param.requires_grad = False
      
  def unfreeze_bert_encoder(self):
    for param in self.bert.parameters():
      param.requires_grad = True
      

       


In [0]:
#from pytorch_transformers import BertConfig

In [0]:
from pytorch_pretrained_bert import BertConfig

In [0]:
#set up model
#bert-base-uncased: hidden_size:768 hidden_layers:12 attention_heads:12
#bert-large-uncased: hidden_size:1024 hidden_layers:24 attention_heads:16

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768, 
                    num_hidden_layers=12, num_attention_heads=12, 
                    intermediate_size=3072)

num_labels = 3
model = BertForSequenceClassification(num_labels=num_labels)


In [28]:
# simple test for model - logits
tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(temp_tokens)])
logits = model(tokens_tensor)
print('logits: ', logits)
print(len(messages))

logits:  tensor([[ 0.7158,  0.4151, -0.7481]], grad_fn=<AddmmBackward>)
3200


In [0]:
context_sentences_count = 1 #meaning above 1 + below 1 + sentence with the drug

# text preprocessing function
def pre_process_text(message, drug_in_context, print_info=False):
  sentences = message.lower().split('.')
  
  indices = set()
  index = 0
  
   
  for sentence in sentences:
    if drug_in_context in sentence:
      indices.add(index)
      #add sentences before
      if index >= 1:
        indices.add(index -1)
      #add sentences afterwards
      if (index < len(sentences) - 1) and index >= 0: 
        indices.add(index+1) 
    index += 1
  
  if print_info:
    print(drug_in_context)
    print(len(sentences))
    print(indices)
   
  
  new_message = ''
  
  
  additional_context = 'About {}. '.format(drug_in_context).lower()
  new_message += additional_context
  
  for ind in indices:
    new_message += ' ' + sentences[ind]
  
  #new_message = new_message.replace(drug_in_context, 'drugincontext')
  
  return new_message
  

In [30]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
print(stop_words_set)
print(len(stop_words_set))

{'itself', 'more', 'whom', 'no', 'do', 'any', 'o', 'other', 'didn', 'but', 'ain', 'and', 'doesn', 'this', 'that', 'on', 'during', 'over', 'few', 'aren', 'our', 'where', 'both', 'should', 'against', 'below', 'it', 'about', 'all', "hadn't", 'further', 'yourself', 'ma', 'him', "she's", 'what', 'above', 'now', 'isn', 'to', 'then', 'are', "should've", 'hers', 'there', "mightn't", 'up', 'before', 'mightn', 'some', 'too', "that'll", 'will', 'because', 'again', 'is', 'doing', 'an', 'does', 'by', 'each', "you've", 's', 'theirs', 'just', 'your', "it's", 'y', 'mustn', 'i', "weren't", 'be', "you'd", 'ourselves', 'haven', 'off', 'wouldn', 'not', 'those', 'you', 'were', "wouldn't", "hasn't", 'than', 'needn', 'with', "haven't", 'don', 'until', 're', 'own', 'they', 'she', 'when', 'wasn', 'if', "mustn't", 'through', 'yourselves', 'here', 'herself', 'we', 'themselves', "shouldn't", 'only', 'll', 'm', 'been', 'himself', 'having', 'how', 'my', 'them', 'can', 'me', 'their', 'has', 'the', 'very', 'most', "i

In [32]:
negative_polarity_stop_words = ['doesn', 'hadn', 'shan', 'nor', 'wasn', 'hasn', "mightn't", "mustn't", "wasn't", 'does', 'don', 'shouldn', 'won', "hasn't", 'couldn', "needn't", "couldn't", "won't", 'not', "hadn't", "wouldn't", "aren't", "weren't", "shan't", "isn't", 'weren', 'wouldn',  "didn't", "don't", "haven't", 'didn', 'neither', 'needn', 'haven', 'isn', 'mustn', 'mightn', "doesn't", 'ain', 'aren', "shouldn't", "never", "neva", "dint", "din", "no"]
negative_polarity_stop_words_set = set(negative_polarity_stop_words)

#remove these from stop words so they can be retained in the text to indicate negative polarity
for wrd in negative_polarity_stop_words:
  if wrd in stop_words_set:
    stop_words_set.remove(wrd)

#arrange more stop words
new_stop_words = []
    
def process_stop_words(stop_word_string, stop_words_set, new_stop_words):
  for wrd in stop_word_string.split(' '):
    if wrd not in stop_words_set:
      new_stop_words.append(wrd)
    
stop_words_new = "a about after all also always am an and any each are at be been being but by can could did do does doing else for from  had has have how i if ill i'm in into is it its i've just like many may me more most much now of only or our some something than that the their them then they thing this to up us very was way we what when where which who why will with without you your youre etc every"
stop_words_days = 'sunday monday tuesday wednesday thursday friday saturday yesterday tomorrow today afternoon morning evening tonight night'
stop_words_months = 'january jan february feb march mar april apr may june jun july jul august aug september sep october oct november nov december dec'
stop_words_numeric = 'one two three four five six seven eight nine ten twenty thirty forty fify hundred hundreds thousand thousands tens first second third fourth fifth sixth seventh eighth ninth tenth eleventh twelfth thirteenth fourteenth fifteenth twentieth hundredth thousandth'

process_stop_words(stop_words_new, stop_words_set, new_stop_words)
process_stop_words(stop_words_days, stop_words_set, new_stop_words)
process_stop_words(stop_words_months, stop_words_set, new_stop_words)
process_stop_words(stop_words_numeric, stop_words_set, new_stop_words)

for wrd in stop_words_set:
  if '\'' in wrd:
    new_stop_words.append(wrd.replace('\'', ''))

for wrd in new_stop_words:
  stop_words_set.add(wrd)
  
print(len(stop_words_set))

239


In [33]:
#import lemmatizer
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
print(lemmatizer.lemmatize('corpora'))

corpus


In [34]:
#deal with word frequency
from collections import Counter
vocab = Counter()
for message in messages:
  words = message.split(' ')
  for word in words:
    if len(word) > 1:
      vocab.update([word])
      
print(vocab.most_common(100))
print(len(vocab.most_common()))

[('the', 31924), ('to', 23714), ('of', 22949), ('and', 22814), ('in', 14950), ('for', 10863), ('is', 10538), ('with', 10038), ('that', 9248), ('or', 6748), ('on', 6677), ('have', 6256), ('was', 6214), ('my', 5975), ('it', 5713), ('be', 5448), ('you', 5237), ('as', 4891), ('are', 4868), ('not', 4585), ('this', 4187), ('at', 3715), ('but', 3662), ('your', 3654), ('The', 3599), ('had', 3201), ('from', 3142), ('patients', 3047), ('has', 2797), ('if', 2739), ('an', 2696), ('may', 2682), ('by', 2670), ('been', 2657), ('can', 2459), ('about', 2413), ('so', 2343), ('treatment', 2337), ('MS', 2337), ('will', 2283), ('more', 2238), ('me', 2203), ('other', 2061), ('all', 1980), ('were', 1938), ('which', 1866), ('after', 1836), ('they', 1725), ('cancer', 1716), ('get', 1623), ('no', 1605), ('any', 1589), ('also', 1571), ('one', 1559), ('It', 1553), ('like', 1549), ('when', 1545), ('some', 1540), ('would', 1532), ('who', 1488), ('there', 1487), ('just', 1457), ('do', 1443), ('first', 1428), ('very'

In [35]:
#words with frequency above the threshold
count = 0
word_freq_threshold = 5
for word in vocab:
  if vocab[word] >= word_freq_threshold:
    count += 1

print(count)

14763


In [0]:
# pre processing tokens. 
# returns token ids 
def process_tokens(text, tokenizer, max_seq_length = 256, return_tensor=False, printInfo=False):
  #remove symbols from text
  text = remove_symbols(text)
  #tokenize the plain text 
  tokenized_text = tokenizer.tokenize(text)
  #remove stop words
  tokenized_text = remove_stopwords(tokenized_text)
  
  #clip data that is more than max length
  if len(tokenized_text) > max_seq_length:
    tokenized_text = tokenized_text[:max_seq_length]
    
  if printInfo:
    print(tokenized_text)
    
  #convert the tokens to ids
  ids_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    
  #pad the data ids with zeros if the length was less than max seq length
  if len(ids_text) < max_seq_length:
    padding_array = [0] * (max_seq_length - len(ids_text))
    ids_text += padding_array
    
  if return_tensor:
    return torch.tensor(ids_text)
  
  return ids_text

def remove_stopwords(tokenized_text):
  result = []
  for token in tokenized_text:
    if token in stop_words_set:
      continue
    '''if token in negative_polarity_stop_words_set:
      result.append("negative")'''
    '''if (token not in vocab) or (vocab[token] >= word_freq_threshold):
      result.append(token)'''
    result.append(token)
  
  return result

def remove_symbols(text):
  #remove urls
  text = re.sub(r'https?:\/\/[\w\d\.\/\-\?\=#]*', '', text)
  #remove numbers
  text = re.sub(r'\d', '', text)
  #removes symbols except . ? ! [] -
  text = re.sub(r'[^!\?\-\.\w_\s]+','', text)
  #text = re.sub(r'[~`"@#$%{}:;,\+\(\)\*&<>]', '', text) 
  return text

In [37]:
#test support methods
test_str = 'this is a review about ocrelizumab!. he + means * drug, that, & everyone is buzzing about in the ms circles (who knew there were ms circles? well, there are ); the new drug should be approved at the end of march, so my last tysabri infusion will be in march i think we want to put you on the new '
test_tokens_1 = test_str.split(' ')
print(remove_stopwords(test_tokens_1))
print(remove_symbols(test_str))


['review', 'ocrelizumab!.', '+', 'means', '*', 'drug,', 'that,', '&', 'everyone', 'buzzing', 'ms', 'circles', '(who', 'knew', 'ms', 'circles?', 'well,', ');', 'new', 'drug', 'approved', 'end', 'march,', 'last', 'tysabri', 'infusion', 'think', 'want', 'put', 'new']
this is a review about ocrelizumab!. he  means  drug that  everyone is buzzing about in the ms circles who knew there were ms circles? well there are  the new drug should be approved at the end of march so my last tysabri infusion will be in march i think we want to put you on the new 


In [38]:
#test preprocessing function
ind = 10
test_message = pre_process_text(messages[ind], drug_info[ind], True)
print(test_message)
tokens_output = process_tokens(test_message, tokenizer, return_tensor=True, printInfo=True)
print(tokens_output)

gilenya
22
{0, 1, 2, 3, 4, 5, 16, 17, 18}
about gilenya.  from ms news today, august 29, 2016:  quote gilenya-like therapy shows benefit in secondary progressive ms patients in phase 3 trial ines martins, phd patients with secondary progressive multiple sclerosis (spms) who were treated with baf312 (siponimod), a sphingosine-1-phosphate (s1p) inhibitor, in a phase 3 clinical trial showed a significantly reduced risk for disability progression compared to placebo, novartis recently announced  baf312 is a selective modulator of specific types of the s1p receptor  this receptor is commonly found at the surface of immune cells that infiltrate and induce damage in the central nervous system (cns), leading to loss of function in ms patients  similar to gilenya (fingolimod), baf312 works by binding to these specific receptors and preventing the migration and activation of immune cells in the cns  baf312, however, binds more selectively to specific s1p receptors than gilenya, particularly to s

In [39]:
# sample pipeline of text 
test_tokens = tokenizer.tokenize(test_message)
test_tokens_ids = tokenizer.convert_tokens_to_ids(test_tokens)

print(test_tokens)
print(test_tokens_ids)
print(len(test_tokens_ids))

test_tokens_tensor = torch.tensor([test_tokens_ids])
print('is tensor: ', type(test_tokens_tensor) == type(torch.Tensor()))
print(test_tokens_tensor)

logits = model(tokens_tensor)
print(logits)

result = F.softmax(logits, dim=1)
print(result)


['about', 'gil', '##en', '##ya', '.', 'from', 'ms', 'news', 'today', ',', 'august', '29', ',', '2016', ':', 'quote', 'gil', '##en', '##ya', '-', 'like', 'therapy', 'shows', 'benefit', 'in', 'secondary', 'progressive', 'ms', 'patients', 'in', 'phase', '3', 'trial', 'in', '##es', 'martins', ',', 'phd', 'patients', 'with', 'secondary', 'progressive', 'multiple', 'sc', '##ler', '##osis', '(', 'sp', '##ms', ')', 'who', 'were', 'treated', 'with', 'ba', '##f', '##31', '##2', '(', 'sip', '##oni', '##mo', '##d', ')', ',', 'a', 'sp', '##hing', '##osi', '##ne', '-', '1', '-', 'phosphate', '(', 's', '##1', '##p', ')', 'inhibitor', ',', 'in', 'a', 'phase', '3', 'clinical', 'trial', 'showed', 'a', 'significantly', 'reduced', 'risk', 'for', 'disability', 'progression', 'compared', 'to', 'place', '##bo', ',', 'nova', '##rti', '##s', 'recently', 'announced', 'ba', '##f', '##31', '##2', 'is', 'a', 'selective', 'mod', '##ulator', 'of', 'specific', 'types', 'of', 'the', 's', '##1', '##p', 'receptor', 'thi

In [40]:

#preprocess all the messages (texts)
new_messages = []
index = 0
for message in messages:
  msg = pre_process_text(message, drug_info[index])
  msg = process_tokens(msg, tokenizer, return_tensor=True)
  new_messages.append(msg)
  index += 1
  
print(len(new_messages))
assert(len(new_messages) == len(messages))

3200


In [0]:
#splitting input data for training and testing
from sklearn.model_selection import train_test_split
#X = np.array(new_messages)
X = new_messages
Y = np.array(sentiments)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

In [0]:
#changing Y to one-hot encoded values
#pd.get_dummies returns data frames. 
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [0]:
max_seq_length = 256
temp_tensor_type = type(torch.Tensor())

#class that will hold training data
class TextDataset(Dataset):
  def __init__(self, x_y_data, transform=None):
    self.x_y_data = x_y_data
    self.transform = transform
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
  def __getitem__(self, index):
    #prepare input
    input_item = self.x_y_data[0][index]
    input_tensor = input_item 
    '''if type(input_item) != temp_tensor_type:
      ids_text = process_tokens(input_item, self.tokenizer, 
                              max_seq_length)
      input_tensor = torch.tensor(ids_text)'''
    
    #prepare output
    output = self.x_y_data[1][index]
    output_tensors = [torch.from_numpy(output)]  
    
    return input_tensor, output_tensors[0]
  
  def __len__(self):
    return len(self.x_y_data[0])
      


In [0]:
#form datasets
batch_size = 16

train_lists = [X_train, Y_train]
test_lists = [X_test, Y_test]

training_dataset = TextDataset(x_y_data = train_lists)
test_dataset = TextDataset(x_y_data= test_lists)

In [45]:
#form data loaders dicts for training loops
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=False, num_workers = 0),
                   'val': torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 0)}

dataset_sizes = {'train' : len(train_lists[0]),
                'val' : len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [46]:
dataset_sizes['val']

640

In [0]:
logits_arr = []
labels_arr = []
logits_val_arr = []
#train method
def train(model, criterion, optimizer, scheduler, num_epochs = 25):
  start_time = time.time()
  print('starting training process. time: ', start_time)
  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 100
  best_acc = 0.0
  
  #for every epoch
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('---------')
    
    global labels_arr, logits_arr, logits_val_arr
    labels_arr, logits_arr, logits_val_arr = [], [], []
    
    #for every phase - train & validation
    for phase in ['train', 'val']:
      
      if phase == 'train':
        scheduler.step()
        model.train()
      else:
        #set model to eval state in validation mode
        model.eval()
        
      running_loss = 0.0
      accuracy = 0
      match_count = 0
      
      for features, labels in dataloaders_dict[phase]:
        features = features.to(device)
        labels = labels.to(device)
        
        #start from zero grad
        optimizer.zero_grad()
        
        #track history only in training phase
        with torch.set_grad_enabled(phase == 'train'):
          logits = model(features)
          logits = F.softmax(logits, dim=1)
          
          #[0] gives max vals, [1] gives max indices, max(dim=1) for horizontal
          labels_max_indices = torch.max(labels.float(), 1)[1] 
          
          loss = criterion(logits, labels_max_indices)
          
          #do backprop only in training phase
          if phase=='train':
            loss.backward()
            optimizer.step()
            
        #metrics
        running_loss += loss.item() * features.size(0)
        max_label_indices = torch.max(labels, 1)[1]
        max_logit_indices = torch.max(logits, 1)[1]
        matches = (max_label_indices == max_logit_indices)
        match_count += torch.sum(matches)
        
        logits_arr.append(max_logit_indices)
        labels_arr.append(max_label_indices)
        
        if phase=='val':
          logits_val_arr.append(logits)
        
      epoch_loss = running_loss / dataset_sizes[phase]
      accuracy = match_count.double() / dataset_sizes[phase]
        
      print('{} total loss: {:.4f}'.format(phase, epoch_loss))
      print('{} ACCURACY: {:.4f}'.format(phase, accuracy))
      
      #see if this is the best from last time
      if phase=='val' and epoch_loss < best_loss:
        print('saving with loss of {}'.format(epoch_loss))
        print('improved over previous best: {}'.format(best_loss))
        best_loss = epoch_loss
        best_acc = accuracy
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'bert_model_state_text_classific.pth')
        
  time_elapsed = time.time() - start_time
  print('training completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, 
                                                       time_elapsed % 60))
  print('best val acc.: {:.4f}, val loss: {:.4f}'.format(best_acc, best_loss))
  
  #return the model with best loss weights
  model.load_state_dict(best_model_wts)
  return model

In [48]:
#move model to device (gpu if any)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [0]:
# learning rate settings
lrLast = 0.001
lrMain = 0.00001
optimizer = optim.Adam([{"params": model.bert.parameters(), "lr": lrMain},
                       {"params": model.classifier.parameters(), "lr": lrLast}])
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [0]:
# execute training
model_ft = train(model, criterion, optimizer, exp_lr_scheduler, num_epochs=20)

starting training process. time:  1568405264.258274
Epoch 0/19
---------


In [0]:
#save final model (best selected) to disk
torch.save(model_ft.state_dict(), 'drive/My Drive/Colab Notebooks/DrugData/bert_model_state_text_classific_best.pth')

In [0]:
torch.cuda.empty_cache() 

In [0]:
#checking percentage of prediction for each label
counts = {}
corrects = {}
for index in range(len(labels_arr)):
  inner_index = 0
  for item in labels_arr[index]:
    item = int(item)
    if item in counts:
      counts[item] += 1
    else:
      counts[item] = 1
      
    if item == int(logits_arr[index][inner_index]):
      if item in corrects:
        corrects[item] += 1
      else:
        corrects[item] = 1
    inner_index += 1


In [0]:
print(counts)
print(corrects)

In [0]:
results = []
outer_index = 160
for batch_arr in logits_val_arr:
  inner_index = 0
  for arr in batch_arr:
    li = []
    for item in arr:
      li.append(float(item))#logit values
    li.append(int(labels_arr[outer_index][inner_index]))#original
    li.append(int(logits_arr[outer_index][inner_index]))#predicted
    results.append(li)
    inner_index +=1
  outer_index += 1
  
print(len(results))

In [0]:
len(labels_arr)

In [0]:
np.savetxt('drive/My Drive/Colab Notebooks/DrugData/model_labels_vs_logits_3200_1.csv', np.array(results), delimiter=",")